In [20]:
pip install influxdb_client influxdb


  Obtaining dependency information for influxdb_client from https://files.pythonhosted.org/packages/fa/c1/b508d7044ce620c01ea7a29c195b80977fc8ae6c362751a019d7cd199b98/influxdb_client-1.39.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/63.9 kB ? eta -:--:--
     ------------ ------------------------- 20.5/63.9 kB 682.7 kB/s eta 0:00:01
     ------------------------------------ - 61.4/63.9 kB 825.8 kB/s eta 0:00:01
     -------------------------------------- 63.9/63.9 kB 688.2 kB/s eta 0:00:00
     ---------------------------------------- 0.0/77.9 kB ? eta -:--:--
     ---------------------------------------- 77.9/77.9 kB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/217.8 kB ? eta -:--:--
     -------------------------------------- 217.8/217.8 kB 6.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/744.0 kB ? eta -:--:--
   --------------------------------------- 744.0/744.0 kB 23.7 MB/s eta 0:00:00


In [24]:
from kafka import KafkaConsumer
from json import loads
import json
from time import sleep
import pandas as pd
from pyspark.sql import SparkSession
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

In [37]:
token = "En1iX5zqnyR_AT71S6Ahz8_Hs78nrJHwEkZDDksf4J6reHJNqXzbaMEXbmBjy7I-bdzp2k8fy7E1FjU1f2ZWsA=="
org = "mema_org"
bucket = "mema_bucket"
url = "http://localhost:8086"

In [38]:
spark = SparkSession.builder.appName("mema").getOrCreate()
consumer = KafkaConsumer('hai-input',bootstrap_servers=['localhost:9092'],
                         group_id='albert-group',
                         auto_offset_reset='earliest', 
                        enable_auto_commit=True,
                        value_deserializer=lambda x: loads(x.decode('utf-8')))

In [40]:
with influxdb_client.InfluxDBClient(url=url, token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    for message in consumer:
        message = message.value
        if len(message) < 10:
            break
        try:
            point = json.dumps(message)
            print(point)
            print(type(point))
            point = eval(point)
            point_values = []
            for key in point:
                print(key)
                print(point[key])
                p = influxdb_client.Point("point").field(key, point[key])
                write_api.write(bucket, org, p)
                point_values.append(point[key])
        except NotImplementedError:
            pass

{"timestamp": "2022-08-12 16:00:01", "P1_FCV01D": 60.99316, "P1_FCV01Z": 60.495, "P1_FCV02D": 15.0, "P1_FCV02Z": 12.44507, "P1_FCV03D": 50.53432, "P1_FCV03Z": 51.99585, "P1_FT01": 64.69727, "P1_FT01Z": 336.85709, "P1_FT02": 2387.84766, "P1_FT02Z": 3116.42798, "P1_FT03": 237.50301, "P1_FT03Z": 980.94177, "P1_LCV01D": 19.11313, "P1_LCV01Z": 18.60199, "P1_LIT01": 395.39975, "P1_PCV01D": 26.19906, "P1_PCV01Z": 26.84173, "P1_PCV02D": 12.0, "P1_PCV02Z": 11.68976, "P1_PIT01": 0.94315, "P1_PIT01_HH": 524320, "P1_PIT02": 0.40756, "P1_PP01AD": 0, "P1_PP01AR": 0, "P1_PP01BD": 1, "P1_PP01BR": 1, "P1_PP02D": 1, "P1_PP02R": 1, "P1_PP04": 33.82419, "P1_PP04D": 1, "P1_PP04SP": 27.48882, "P1_SOL01D": 524320, "P1_SOL03D": 524320, "P1_STSP": 1, "P1_TIT01": 29.21143, "P1_TIT02": 33.86535, "P1_TIT03": 26.7478, "P2_24Vdc": 28.02821, "P2_ATSW_Lamp": 1, "P2_AutoGO": 1, "P2_AutoSD": 119.04872, "P2_Emerg": 0, "P2_MASW": 0, "P2_MASW_Lamp": 0, "P2_ManualGO": 0, "P2_ManualSD": 12.92742, "P2_OnOff": 1, "P2_RTR": 28

In [16]:
class HaiConsumer:
    def __init__(self, topic, bootstrap_servers):
        self.topic = topic
        self.bootstrap_servers = bootstrap_servers
        self.consumer = KafkaConsumer(
            self.topic,
            bootstrap_servers=self.bootstrap_servers,
            auto_offset_reset='earliest',
            enable_auto_commit=True,
            value_deserializer=lambda x: loads(x.decode('utf-8')))

    def consume(self):
        counter = 0
        # write to file
        with open('data.txt', 'w') as f:
            for i, message in enumerate(self.consumer):
                message = message.value
                # convert dict to string
                message = str(message)
                print(message)
                print(counter)
                counter += 1
                f.write(message)
                if i > 100:
                    break

In [17]:
topic = 'hai-input'
bootstrap_servers = ['localhost:9092']
consumer = HaiConsumer(topic, bootstrap_servers)
consumer.consume()

{'timestamp': '2022-08-12 16:00:01', 'P1_FCV01D': 60.99316, 'P1_FCV01Z': 60.495, 'P1_FCV02D': 15.0, 'P1_FCV02Z': 12.44507, 'P1_FCV03D': 50.53432, 'P1_FCV03Z': 51.99585, 'P1_FT01': 64.69727, 'P1_FT01Z': 336.85709, 'P1_FT02': 2387.84766, 'P1_FT02Z': 3116.42798, 'P1_FT03': 237.50301, 'P1_FT03Z': 980.94177, 'P1_LCV01D': 19.11313, 'P1_LCV01Z': 18.60199, 'P1_LIT01': 395.39975, 'P1_PCV01D': 26.19906, 'P1_PCV01Z': 26.84173, 'P1_PCV02D': 12.0, 'P1_PCV02Z': 11.68976, 'P1_PIT01': 0.94315, 'P1_PIT01_HH': 524320, 'P1_PIT02': 0.40756, 'P1_PP01AD': 0, 'P1_PP01AR': 0, 'P1_PP01BD': 1, 'P1_PP01BR': 1, 'P1_PP02D': 1, 'P1_PP02R': 1, 'P1_PP04': 33.82419, 'P1_PP04D': 1, 'P1_PP04SP': 27.48882, 'P1_SOL01D': 524320, 'P1_SOL03D': 524320, 'P1_STSP': 1, 'P1_TIT01': 29.21143, 'P1_TIT02': 33.86535, 'P1_TIT03': 26.7478, 'P2_24Vdc': 28.02821, 'P2_ATSW_Lamp': 1, 'P2_AutoGO': 1, 'P2_AutoSD': 119.04872, 'P2_Emerg': 0, 'P2_MASW': 0, 'P2_MASW_Lamp': 0, 'P2_ManualGO': 0, 'P2_ManualSD': 12.92742, 'P2_OnOff': 1, 'P2_RTR': 28